In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd


import random



N=9
nowcast=1

y_input_size=1
z_input_size=1

file="india_segmented.csv"

series=read_csv(file, sep=",", header=0,parse_dates=True)
series=series.values
series


array([[0, '2020-05-15 00:00:00', 0.0, 25.18431613333333, 0.0],
       [1, '2020-05-15 00:15:00', 0.0, 25.08458866666667, 0.0],
       [2, '2020-05-15 00:30:00', 0.0, 24.935752600000004, 0.0],
       ...,
       [3261, '2020-06-17 23:15:00', 0.0, 22.008274642857145, 0.0],
       [3262, '2020-06-17 23:30:00', 0.0, 21.96949473333333, 0.0],
       [3263, '2020-06-17 23:45:00', 0.0, 21.90928766666667, 0.0]],
      dtype=object)

In [2]:

X1=sensors=np.array(series[:,2])
X2=sensors=np.array(series[:,3])
Y0=sensors=np.array(series[:,4])

print(X1[0])
print(X2[0])
print(Y0[0])

0.0
25.18431613333333
0.0


In [3]:
import tensorflow as tf 

X1n=tf.keras.utils.normalize(    X1, axis=-1, order=2)
X1n=np.array(X1n[0])
#X1n=X1


print(X1n)
print(X1n.shape)

X2n=tf.keras.utils.normalize(X2, axis=-1, order=2)
X2n=np.array(X2n[0])
#X2n=X2
print(X2n)
print(X2n.shape)

X0=np.zeros((len(X1n),2))

X0[:,0]=X1n
X0[:,1]=X2n

Y=np.zeros((len(Y0),1))
Y[:,0]=Y0

print(X0)
print(X0.shape)
print(Y.shape)
print(Y)

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
(3264,)
[0.0171390232809216 0.017071154399197833 0.01696986497771283 ...
 0.014977588808837745 0.014951197392503793 0.01491022386311301]
(3264,)
[[0.         0.01713902]
 [0.         0.01707115]
 [0.         0.01696986]
 ...
 [0.         0.01497759]
 [0.         0.0149512 ]
 [0.         0.01491022]]
(3264, 2)
(3264, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [4]:
X=[]
for i in range(N,len(X0)):
    X.append(X0[i-N:i])
X=np.array(X)    
print(X.shape)

Y=Y[N-nowcast:len(Y)-nowcast]
print(Y.shape)



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
print(Y[40])
scaler.fit(Y)

Y = scaler.transform(Y)
print(Y[40])

(3255, 9, 2)
(3255, 1)
[7931.5]
[0.55415764]


In [5]:
print(X[0])
print(Y[0])

[[0.         0.01713902]
 [0.         0.01707115]
 [0.         0.01696986]
 [0.         0.01690887]
 [0.         0.01675602]
 [0.         0.01669788]
 [0.         0.01676769]
 [0.         0.01692717]
 [0.         0.01697067]]
[0.]


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X , Y ,test_size=0.2, random_state=0)

In [7]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K


mae = tf.keras.losses.MeanAbsoluteError()

def loss_error_gen(y_true, y_pred):
        return tf.keras.metrics.mean_squared_error(y_true, y_pred)

    
def loss_error_dis(y_true, y_pred):
        return tf.keras.metrics.mean_squared_error(y_true, y_pred)

def loss_error_lier(y_true, y_pred):    
        return tf.keras.metrics.mean_squared_error(y_true, y_pred)
    
def loss_error_gan(y_true, y_pred):
        
        return tf.keras.metrics.mean_squared_error(y_true, y_pred)

    

def make_generator():
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    concat = Dense(256)(latent)
    concat = Dense(128)(concat)
    prediction = Dense(1,activation="linear")(concat)
    model = Model(inputs=[x], outputs=prediction)
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model   


def make_lier():
    
    label = Input(shape=(y_input_size,))
    label_output = Dense(128, activation='relu')(label)
    label_output = Dense(64, activation='relu')(label_output)
    
    noise = Input(shape=(z_input_size,))
    noise_output = Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.Zeros(), bias_initializer='zeros')(noise)

    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    latent = concatenate([label, noise_output,latent])
    
    concat = Dense(256)(latent)
    concat = Dense(128)(concat)
    prediction = Dense(1,activation="linear")(concat)
    
    concat2 = Dense(256)(latent)
    concat2 = Dense(128)(concat2)
    inc_fake = Dense(1,activation="linear")(concat2)

    model = Model(inputs=[label,noise, x], outputs=[prediction,inc_fake])

    return model 



def make_discriminator():
    
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    flat = Flatten()(x_output)

    label = Input(shape=(y_input_size,))
    label_output = Dense(128, activation='relu')(label)
    label_output = Dense(64, activation='relu')(label_output)

    latent = concatenate([flat, label_output])

    
    concat = Dense(256)(latent)
    concat = Dense(128)(concat)
    prediction = Dense(1,activation="linear")(concat)
    
    concat2 = Dense(256)(latent)
    concat2 = Dense(128)(concat2)
    inc = Dense(1,activation="linear")(concat2)

    model = Model(inputs=[x, label], outputs=[prediction, inc])
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.summary()
    return model  

x = Input(shape=(N,2))
generator = make_generator()
prediction = generator([x])



lier = make_lier()
noise = Input(shape=(z_input_size,))
[prediction2, inc_fake] = lier([prediction,noise,x])

discriminator = make_discriminator()

[prediction3, inc] = discriminator([x, prediction2])

discriminator.trainable = False

combined = Model([noise, x], [prediction,prediction2,prediction3, inc_fake, inc])
combined.compile(loss='mean_squared_error',optimizer='adam')
combined.summary()

#combined = Sequential()
#combined.add(generator)
#combined.add(lier)
#combined.add(discriminator)
#combined.compile(loss='binary_crossentropy', optimizer='adam')
    


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 9, 2)]       0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 8, 16)        80          input_7[0][0]                    
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 6, 32)        1568        conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 4, 64)        6208        conv1d_7[0][0]                   
____________________________________________________________________________________________

In [8]:
print(x_train.shape)
print(y_train.shape)



#history = generator.fit(x_train,y_train ,
#                    batch_size=64,
#            validation_data=(x_val,y_val),
#            epochs=40)

(2604, 9, 2)
(2604, 1)


In [9]:
#from sklearn.metrics import confusion_matrix
#yhat = generator.predict(x_val)
#yhat=scaler.inverse_transform(yhat).flatten()
#y_val0=scaler.inverse_transform(y_val).flatten()
#print(y_val)
#print(yhat)

#file = open('prediction.csv', 'w')
#file.write('ground truth\tprediction\n')
#for i in range(0,len(y_val)):
#    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
#file.close()

#print("RMSE")
#error=tf.keras.metrics.mean_squared_error(
#    y_val0, yhat
#)
#print(np.sqrt(error))

In [10]:
def generate_true_samples(batch_size):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x, true_labels = x_train[idx], y_train[idx]
    return x,true_labels

def getNoise():
    return  np.random.normal(0, 1, (batch_size, z_input_size))



In [11]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

    
n_epochs=80
batch_size=64

zeros = np.zeros((batch_size, 1))

file = open('metrics_cgan_reg_inc.csv', 'w')

for epoch in range(n_epochs):
    print("e:",epoch)
    for batch_idx in range(int(x_train.shape[0] // batch_size)):
        
        
        x,true_labels = generate_true_samples(batch_size)
        #generator.train_on_batch([x],true_labels)
        prediction = generator.predict([x])

        noise = getNoise()
        
        [prediction2,inc_fake] = lier.predict([prediction,noise,x])
        
        zeros_inc=zeros
        inc_fake2=inc_fake
        
        #discriminator.train_on_batch([x, prediction], [prediction,zeros_inc])
        discriminator.train_on_batch([x, prediction-inc_fake2], [prediction+inc_fake2,+inc_fake2])
        discriminator.train_on_batch([x, prediction+inc_fake2], [prediction-inc_fake2,-inc_fake2])
        #discriminator.train_on_batch([x, prediction2], [prediction-inc_fake2,-inc_fake2])
        
        #combined.train_on_batch([noise,x], [true_labels-inc_fake2,zeros,-inc_fake2])
        
        x,true_labels = generate_true_samples(batch_size)
        #prediction = generator.predict([x])
        #[prediction2,inc_fake] = lier.predict([prediction,noise,x])
        #inc_fake2=0.25*np.random.normal(0, 1, (batch_size, 1))
        
        
        combined.train_on_batch([noise,x], [true_labels,true_labels,true_labels,zeros_inc,zeros_inc])
        
        #d_loss_fake = discriminator.train_on_batch([x, predicted_labels_inc], inc_fake_labels)
        #d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        


        #lier.trainable = False
        #x2,_ = generate_true_samples(batch_size)
        #g_loss = combined.train_on_batch([noise,x], inc_fake_labels)
        #g_loss = combined.train_on_batch([noise,x], inc_fake_labels)
    
    #pf = discriminator.predict([x, predicted_labels])        
    #pt = discriminator.predict([x, predicted_labels_inc])
    #pc = combined.predict([noise,x2])

    #print('fake:',pf)    
    #print('true:',pt)    
    #print('combined:',pc)


    noise = np.random.normal(0, 1, (x_val.shape[0], z_input_size))
    print('noise:',noise)
    yhat = generator.predict([x_val])
    [yhat2,inc] = lier.predict([yhat,noise, x_val])
    [yhat3,dif]=discriminator.predict([x_val, yhat+inc])


    print("RMSE1:")

    error=tf.keras.metrics.mean_squared_error(
        y_val.flatten(), yhat.flatten()
    )
    print(np.sqrt(error))
    file.write(str(np.sqrt(error))+'\t')
   
    yhat0=scaler.inverse_transform(yhat).flatten()
    y_val0=scaler.inverse_transform(y_val).flatten()

    print("RMSE2:")
    error0=tf.keras.metrics.mean_squared_error(
        y_val0, yhat0
    )
    print(np.sqrt(error0))    
    file.write(str(np.sqrt(error0))+'\t')
    
    yhat2=scaler.inverse_transform(yhat2).flatten()
    yhat3=scaler.inverse_transform(yhat3).flatten()
    
    
    file.write(str(np.mean(yhat0))+'\t')
    file.write(str(np.mean(yhat2))+'\t')
    file.write(str(np.mean(yhat3))+'\n')

file.close() 
    
    #print("g_loss:",g_loss)
    #print("d_loss:",d_loss)

e: 0
noise: [[-8.91250548e-01]
 [ 7.97643870e-01]
 [ 1.30440838e+00]
 [ 1.17149576e+00]
 [ 4.06099448e-01]
 [-1.55614596e+00]
 [-1.03428400e-01]
 [ 7.30158086e-01]
 [ 3.68133407e-01]
 [-6.60281902e-01]
 [-1.68439939e+00]
 [-6.28095396e-01]
 [-1.67169862e+00]
 [ 9.55963274e-01]
 [-8.74273438e-01]
 [ 3.05898119e+00]
 [-8.12385611e-01]
 [-5.37690123e-01]
 [-1.21775547e+00]
 [-1.51380701e+00]
 [ 3.24205167e-01]
 [ 7.66090348e-01]
 [ 6.71205056e-01]
 [ 8.33062556e-01]
 [ 1.73618395e+00]
 [ 1.30454105e+00]
 [-4.09112960e-01]
 [ 1.25583479e+00]
 [-1.36317376e+00]
 [-1.48205528e-01]
 [ 3.94173074e-01]
 [-2.03159894e+00]
 [ 1.03077464e+00]
 [-4.87611310e-01]
 [-1.65223279e-01]
 [ 8.12470128e-01]
 [-1.71868033e-01]
 [-7.38425097e-01]
 [ 1.70117475e-01]
 [-7.01054430e-01]
 [-9.05053715e-01]
 [-4.42155906e-01]
 [-7.83231640e-01]
 [-1.60534788e+00]
 [-6.14391887e-01]
 [-7.58769393e-01]
 [-2.07726331e+00]
 [-7.22822803e-01]
 [ 1.21968862e+00]
 [ 8.26029021e-01]
 [-4.63967951e-01]
 [ 2.44173660e-01]


RMSE1:
0.13307673
RMSE2:
1904.6891
e: 1
noise: [[-1.16301645]
 [ 0.59044108]
 [-1.28585648]
 [ 0.95164465]
 [ 0.59377942]
 [-1.50200106]
 [ 0.44027734]
 [ 2.2519788 ]
 [ 0.50434413]
 [ 0.74824361]
 [ 0.86009878]
 [-1.74495881]
 [ 0.36735751]
 [-0.32927778]
 [-0.74976771]
 [ 0.53001052]
 [-0.21357766]
 [-2.13444317]
 [-0.10748862]
 [ 0.28205898]
 [ 1.6089128 ]
 [-1.78482683]
 [-1.31404555]
 [-1.44299122]
 [ 0.16310555]
 [-0.35099165]
 [ 0.45140694]
 [ 0.44321678]
 [ 0.19701888]
 [-0.51521355]
 [ 0.89252092]
 [-0.4305571 ]
 [-0.73427559]
 [-1.44087433]
 [-0.05346504]
 [-0.41463292]
 [-0.76996835]
 [-0.23330643]
 [-0.729862  ]
 [-0.15187306]
 [-0.37428637]
 [ 0.14621428]
 [-1.69812916]
 [-0.9174678 ]
 [ 0.10978849]
 [ 0.81282614]
 [-1.27231013]
 [-3.65208212]
 [ 0.58467565]
 [-0.91576937]
 [-0.03427949]
 [ 0.78663378]
 [ 0.23837192]
 [-1.76647588]
 [-0.93152429]
 [-0.19848708]
 [ 0.46524225]
 [-0.34970703]
 [ 1.20049607]
 [ 0.48650843]
 [ 0.54838263]
 [ 0.34080238]
 [-1.33680145]
 [ 0.308

0.13014424
RMSE2:
1862.7173
e: 2
noise: [[-8.85101186e-01]
 [ 1.05437249e+00]
 [ 5.83846882e-01]
 [-5.59260708e-01]
 [-2.69576535e-01]
 [-2.77569026e-01]
 [ 1.20832033e+00]
 [-5.44957237e-01]
 [-1.96963001e-01]
 [-3.97167074e-01]
 [-1.45369960e+00]
 [-1.85367174e+00]
 [-5.01482343e-01]
 [ 4.11671310e-01]
 [-6.22178392e-01]
 [-4.82234394e-01]
 [ 5.49067729e-01]
 [-1.53205761e+00]
 [-6.96979753e-01]
 [-6.99731137e-01]
 [ 1.61077696e+00]
 [ 6.07748156e-01]
 [ 4.04856603e-01]
 [ 1.61670503e+00]
 [-8.18304124e-01]
 [ 9.62382537e-01]
 [-7.41848092e-01]
 [-1.50569699e+00]
 [ 9.89527505e-01]
 [-7.41176096e-01]
 [-2.13832636e-01]
 [ 2.26513686e-01]
 [-8.79902072e-01]
 [-4.04105820e-01]
 [ 8.43986418e-01]
 [-2.06281497e-01]
 [ 3.61297731e-02]
 [ 1.34537965e+00]
 [ 6.93668115e-01]
 [-2.15859063e+00]
 [-3.25830000e-01]
 [-6.10683774e-01]
 [ 8.02414153e-01]
 [ 6.70631778e-01]
 [ 7.52638335e-01]
 [ 1.74028066e-02]
 [-4.64449742e-01]
 [-1.01724356e-01]
 [-9.26667066e-01]
 [-6.56240027e-01]
 [-5.97772

0.11637073
RMSE2:
1665.5809
e: 3
noise: [[-1.23632662e+00]
 [-3.64349830e-01]
 [-1.57393420e+00]
 [-1.76288978e+00]
 [-3.12531429e-01]
 [-2.26230593e-01]
 [ 9.21785071e-01]
 [ 4.73084994e-01]
 [ 5.49400990e-01]
 [ 1.74258639e+00]
 [-2.25842344e-01]
 [-2.70622615e+00]
 [-7.07352222e-02]
 [-5.84994441e-01]
 [-8.27886884e-01]
 [-5.48183171e-01]
 [ 1.79610742e-01]
 [-8.03298023e-01]
 [ 2.99411550e-01]
 [-2.57654835e+00]
 [ 4.22589863e-01]
 [ 1.95840310e+00]
 [-8.71312153e-01]
 [-2.57817118e-01]
 [-6.46872653e-01]
 [-1.34643355e+00]
 [ 2.24923009e-01]
 [-8.82854121e-02]
 [-4.05123881e-01]
 [ 1.03647270e+00]
 [-5.03816536e-01]
 [ 2.46893507e-01]
 [-5.43280856e-02]
 [ 1.17106359e+00]
 [ 8.35060339e-01]
 [-1.00299089e+00]
 [ 4.32310516e-02]
 [ 1.54691715e+00]
 [ 5.38415880e-01]
 [-5.96012610e-01]
 [-8.62262274e-01]
 [ 8.34589924e-01]
 [ 1.67892279e+00]
 [ 3.88495095e-01]
 [ 1.13811730e+00]
 [-8.36327661e-01]
 [-1.43182268e-01]
 [-4.73241537e-02]
 [ 2.08947161e+00]
 [ 2.41857384e-01]
 [ 4.65946

noise: [[-0.80830437]
 [ 0.26198697]
 [ 0.40116506]
 [-2.15231694]
 [-0.00805156]
 [ 0.10311951]
 [ 0.24242119]
 [-0.07987411]
 [ 0.00413531]
 [-0.10880411]
 [ 1.59248434]
 [ 0.25109711]
 [-0.88406656]
 [-0.37545718]
 [ 2.27861562]
 [-0.9448387 ]
 [-0.17259357]
 [-0.26808664]
 [-1.59196821]
 [ 0.24993031]
 [ 0.9868596 ]
 [ 0.11648289]
 [-0.28742477]
 [-1.04517975]
 [ 0.02345626]
 [ 0.34426644]
 [ 1.12366502]
 [ 1.67825279]
 [ 0.62194789]
 [ 0.20960124]
 [ 1.04482602]
 [ 0.97510122]
 [-0.70141903]
 [ 0.20574189]
 [ 0.95928157]
 [-1.08219971]
 [ 0.55141461]
 [-1.02116267]
 [ 0.09529089]
 [-1.40251784]
 [-2.37453033]
 [-0.16353912]
 [ 0.98589946]
 [ 0.12553213]
 [ 1.56781652]
 [ 1.57040249]
 [ 0.84298224]
 [-0.54882698]
 [-0.18476673]
 [-1.4342658 ]
 [ 2.13679083]
 [ 0.27451727]
 [ 0.03894604]
 [ 1.59251975]
 [ 0.38421065]
 [ 1.80880674]
 [ 1.6801518 ]
 [-0.17635802]
 [-0.98565453]
 [-1.65532045]
 [ 0.18651251]
 [ 0.90857432]
 [-1.82254159]
 [ 0.23730786]
 [ 0.45455785]
 [-0.68324986]
 [ 

...

noise: [[ 1.96843028e-01]
 [ 1.84944820e+00]
 [-1.22631492e+00]
 [ 2.18729136e+00]
 [ 5.56162004e-02]
 [-6.38284508e-01]
 [ 8.46427157e-01]
 [-9.92105745e-01]
 [ 1.88822404e+00]
 [-1.45712695e+00]
 [ 2.61301727e-01]
 [-5.23447450e-01]
 [-7.77470131e-01]
 [-7.32683329e-01]
 [ 2.53650864e-01]
 [-1.64456511e-01]
 [ 8.30614482e-01]
 [ 1.24170377e-01]
 [-6.96908327e-01]
 [ 1.28205144e-01]
 [-1.09930897e+00]
 [-1.50287391e-01]
 [-1.34575838e-01]
 [ 2.26045865e-01]
 [ 1.84494634e+00]
 [-4.15927132e-01]
 [-5.52426695e-02]
 [ 2.22417878e+00]
 [ 7.75657883e-01]
 [-1.28772327e+00]
 [-1.53741778e+00]
 [-2.67326900e+00]
 [ 7.69581583e-01]
 [-4.45216539e-01]
 [ 1.17186506e+00]
 [-1.23611954e+00]
 [ 1.11680629e-01]
 [-5.28739712e-01]
 [ 5.67336031e-01]
 [-5.00007357e-01]
 [-1.21986361e+00]
 [ 1.19776917e+00]
 [-2.34286929e+00]
 [ 1.62788169e+00]
 [-1.05000780e+00]
 [-2.46364828e-01]
 [ 1.98874657e+00]
 [ 9.20607930e-01]
 [-7.09799890e-01]
 [ 4.83763236e-01]
 [-1.40003614e+00]
 [ 1.61670998e+00]
 [-1.

0.024208022
RMSE2:
346.48254
e: 77
noise: [[ 8.90037535e-01]
 [ 6.37108972e-01]
 [-2.01680751e+00]
 [-8.11108395e-01]
 [-1.04995933e+00]
 [ 1.80942412e-01]
 [-8.22436920e-01]
 [-7.47457809e-01]
 [ 2.23254460e-01]
 [ 5.29754684e-02]
 [ 2.46338446e-01]
 [ 1.00391147e+00]
 [ 5.99756625e-01]
 [-6.34798499e-01]
 [-3.81787429e-01]
 [-2.01257098e+00]
 [-6.94634747e-01]
 [-1.57296209e-01]
 [-1.64797807e+00]
 [ 9.37205196e-02]
 [-1.33416149e+00]
 [-1.08724480e+00]
 [-1.80978093e+00]
 [ 2.97338704e-01]
 [-1.49104707e+00]
 [ 9.23625814e-01]
 [-5.98952054e-01]
 [-4.04384766e-01]
 [-1.92096161e-01]
 [ 3.97627169e-01]
 [ 7.08863560e-01]
 [-1.49046761e+00]
 [ 2.00606227e-01]
 [-1.90991363e-01]
 [-3.81956010e-01]
 [ 4.32803963e-01]
 [-9.23769192e-01]
 [-1.01111630e+00]
 [ 6.22740599e-01]
 [ 8.81846242e-01]
 [ 8.34405380e-01]
 [-9.56578939e-02]
 [ 1.76227163e-01]
 [ 5.72974827e-01]
 [-8.40133273e-01]
 [-1.17161002e+00]
 [-7.86233234e-02]
 [-2.66949935e-01]
 [ 6.59618724e-01]
 [-2.21575564e-01]
 [-1.014

0.023835868
RMSE2:
341.15598
e: 78
noise: [[ 8.93737385e-01]
 [ 9.71640970e-01]
 [-1.96784116e-01]
 [ 2.68019508e+00]
 [ 3.37534278e-01]
 [-3.69654193e-02]
 [-1.09722365e-01]
 [ 1.86548602e-01]
 [ 5.53123778e-01]
 [-1.46269643e+00]
 [-1.69457851e+00]
 [-8.26446680e-01]
 [-1.11882060e+00]
 [ 1.06517463e+00]
 [ 4.11974204e-01]
 [-1.46832531e+00]
 [ 9.32902377e-01]
 [-7.89600316e-01]
 [-8.05409629e-01]
 [ 9.13285472e-01]
 [ 1.52079361e+00]
 [-7.77565962e-01]
 [-2.59879455e+00]
 [ 5.08259172e-02]
 [ 6.21001900e-01]
 [-1.72638338e-01]
 [-3.60562078e-01]
 [ 1.41788673e+00]
 [-3.14749204e-01]
 [ 5.47804899e-01]
 [ 5.74329918e-01]
 [-4.93841454e-01]
 [ 9.89833182e-01]
 [ 4.49469879e-01]
 [-7.73646205e-01]
 [-5.70947205e-01]
 [ 1.14158382e+00]
 [ 1.23538343e+00]
 [ 1.66683983e+00]
 [ 6.28398977e-01]
 [-6.91575266e-01]
 [ 6.60028932e-01]
 [-2.38303639e-01]
 [-1.30726886e-01]
 [-3.38626902e+00]
 [-8.12159820e-01]
 [-9.73544069e-01]
 [-1.74785872e+00]
 [ 1.51487796e+00]
 [-9.04103815e-01]
 [-2.604

noise: [[-5.73885781e-01]
 [-1.20262303e+00]
 [ 2.77206592e-01]
 [-1.32120655e+00]
 [-8.54900020e-01]
 [ 2.65002657e-01]
 [ 8.20517014e-01]
 [ 2.99912733e-01]
 [ 8.56155805e-01]
 [-7.14548324e-01]
 [ 1.49724170e-01]
 [-4.08050565e-01]
 [ 1.05694517e+00]
 [ 2.70449760e-01]
 [-6.29031706e-01]
 [-6.60453017e-01]
 [ 2.61742233e+00]
 [-2.25703887e+00]
 [ 1.03233319e+00]
 [ 1.82856199e-01]
 [ 2.58208034e-02]
 [ 1.27748735e+00]
 [-5.65825804e-01]
 [-5.80959711e-01]
 [-3.67116806e+00]
 [-1.50681839e+00]
 [ 7.58105701e-01]
 [-1.59664727e+00]
 [ 3.06619162e-02]
 [-2.05435559e+00]
 [-1.13867420e-01]
 [ 2.85060163e-01]
 [ 2.98019094e-01]
 [ 2.33011784e+00]
 [-1.51501223e-01]
 [-1.23193261e+00]
 [-1.65570961e+00]
 [-1.43367428e+00]
 [-4.67833539e-01]
 [-6.87656703e-01]
 [-1.20265496e-01]
 [ 4.95227487e-02]
 [ 5.87558602e-01]
 [ 2.44830380e+00]
 [ 2.21410139e+00]
 [-4.78911612e-02]
 [ 3.23726688e-02]
 [-3.56752297e-02]
 [ 1.47274054e+00]
 [ 1.43663093e+00]
 [-1.41911334e+00]
 [ 5.72897427e-01]
 [-5.

0.026191255
RMSE2:
374.86792


In [12]:
from sklearn.metrics import confusion_matrix



noise = np.random.normal(0, 1, (x_val.shape[0], z_input_size))
print('noise:',noise)
yhat = generator.predict([x_val])
[yhat2,inc] = lier.predict([yhat,noise, x_val])
[yhat3,dif]=discriminator.predict([x_val, yhat+inc])


print("RMSE1:")

error=tf.keras.metrics.mean_squared_error(
    y_val.flatten(), yhat.flatten()
)
print(np.sqrt(error))

#dif2=combined.predict([noise,x_val])        
    
#inc = scaler.inverse_transform(inc).flatten()   
#dif = scaler.inverse_transform(dif).flatten()   


#dif2 = scaler.inverse_transform(dif2).flatten()   

yhat=scaler.inverse_transform(yhat).flatten()
yhat3=scaler.inverse_transform(yhat3).flatten()
yhat2=scaler.inverse_transform(yhat2).flatten()
y_val0=scaler.inverse_transform(y_val).flatten()
inc0=scaler.inverse_transform(inc).flatten()
dif0=scaler.inverse_transform(dif).flatten()
#print(y_val)
#print(yhat)

file = open('prediction_gan.csv', 'w')
file.write('ground truth\tprediction\n')
for i in range(0,len(y_val)):
    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\t"+str(yhat2[i])+"\t"+str(yhat3[i])+"\t"+str(inc0[i])+"\t"+str(-dif0[i])+"\t"+"\n")
file.close()

print("RMSE2:")
error=tf.keras.metrics.mean_squared_error(
    y_val0, yhat
)
#print(np.sqrt(error))
#print(np.mean(yhat), "/", np.std(yhat))
#print(np.mean(yhat2), "/", np.std(yhat2))
#print(np.mean(yhat3), "/", np.std(yhat3))

#print(np.sum(yhat))
#print(np.sum(yhat2))
#print(np.sum(yhat3))

print(np.sqrt(error))



noise: [[ 9.60237769e-01]
 [-1.74155557e+00]
 [-1.14225751e+00]
 [-8.27356892e-01]
 [ 7.70335655e-01]
 [-3.05853531e-01]
 [-8.25720586e-01]
 [-3.17800311e-01]
 [ 8.81579027e-01]
 [-7.82230150e-01]
 [ 7.07260107e-01]
 [-5.38098002e-03]
 [ 1.03124228e+00]
 [ 3.65091260e-01]
 [-2.15696592e-01]
 [-5.85441154e-01]
 [-1.09328891e-01]
 [ 6.09459076e-01]
 [ 3.24119483e-01]
 [-8.59216659e-01]
 [ 1.11590229e+00]
 [ 1.74599311e-01]
 [-1.92772970e-01]
 [ 8.83202220e-01]
 [-1.45122012e+00]
 [ 2.16310077e-01]
 [-7.43856761e-01]
 [-5.46920277e-01]
 [ 4.81375123e-01]
 [-1.29898607e+00]
 [ 8.14921076e-01]
 [ 8.93420487e-01]
 [-1.48874906e+00]
 [-6.47486264e-01]
 [-1.12256348e+00]
 [-6.01083122e-01]
 [-1.42105710e+00]
 [ 4.09319383e-01]
 [-4.36754997e-01]
 [-2.33413005e-01]
 [ 6.54448007e-01]
 [-2.37926011e+00]
 [-9.35959901e-01]
 [-7.96593212e-02]
 [-1.25756183e+00]
 [ 9.24710232e-01]
 [ 8.41113057e-01]
 [-1.37739847e+00]
 [-1.76479836e-01]
 [ 2.18394540e+00]
 [ 6.18090780e-01]
 [-6.51325914e-01]
 [-1.

0.026191255
RMSE2:
374.86792


## 